In [1]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np
import mlflow
from datetime import datetime, timedelta
from alibi_detect.cd import TabularDrift

d:\0Viviana\1 FIAP\5 fase\datathon\venv312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Numba needs NumPy 1.26 or less

In [28]:
DB_PATH = Path().resolve().parent.parent / "app" / "data" / "extraidos" / "dados.db"
print(DB_PATH)

D:\0Viviana\1 FIAP\5 fase\datathon\app\data\extraidos\dados.db


In [29]:
# Conecta ao banco SQLite
conn = sqlite3.connect(DB_PATH)

In [30]:
# 🔍 Consulta os dados
query = """
SELECT 
    infos_basicas_email,
    informacoes_profissionais_area_atuacao,
    cv_pt,
    infos_basicas_data_criacao
FROM applicants
WHERE infos_basicas_data_criacao IS NOT NULL
"""
df = pd.read_sql_query(query, conn)
conn.close()

In [31]:
# Converte a coluna de data
df["infos_basicas_data_criacao"] = pd.to_datetime(df["infos_basicas_data_criacao"], errors="coerce")
df = df.dropna(subset=["infos_basicas_data_criacao"])

# Ordena e calcula limites
data_min = df["infos_basicas_data_criacao"].min()
data_max = df["infos_basicas_data_criacao"].max()

In [32]:
data_max, data_min

(Timestamp('2024-12-02 19:53:32'), Timestamp('2012-12-03 17:21:56'))

In [33]:
# Define o limite como os últimos 20% do intervalo
intervalo_total = data_max - data_min
limite = data_max - intervalo_total * 0.2
print(f"Intervalo: {data_min.date()} → {data_max.date()}")
print(f"Limite automático definido em: {limite.date()}")

Intervalo: 2012-12-03 → 2024-12-02
Limite automático definido em: 2022-07-10


In [34]:
# Divide os dados
df_ref = df[df["infos_basicas_data_criacao"] < limite].drop(columns=["infos_basicas_data_criacao"])
df_cur = df[df["infos_basicas_data_criacao"] >= limite].drop(columns=["infos_basicas_data_criacao"])

In [10]:
df.head(2)

,infos_basicas_email,informacoes_profissionais_area_atuacao,cv_pt,infos_basicas_data_criacao
0,carolina_aparecida@gmail.com,,assistente administrativo\n\n\nsantosbatista\n...,2021-10-11 07:29:49
1,eduardo_rios@hotmail.com,Administrativa,formação acadêmica\nensino médio (2º grau) em ...,2021-10-11 08:56:16


In [12]:
df_ref.head(2)

,infos_basicas_email,informacoes_profissionais_area_atuacao,cv_pt
0,carolina_aparecida@gmail.com,,assistente administrativo\n\n\nsantosbatista\n...
1,eduardo_rios@hotmail.com,Administrativa,formação acadêmica\nensino médio (2º grau) em ...


In [13]:
df_cur.head(2)

,infos_basicas_email,informacoes_profissionais_area_atuacao,cv_pt
1000,juan_azevedo@gmail.com,,resumo\n- experiência profissional e vivência ...
1001,sr._luigi_monteiro@hotmail.com,,front end senior\n


In [35]:
# Codifica texto como números (necessário para Alibi Detect)
df_all = pd.concat([df_ref, df_cur])
df_encoded = df_all.apply(lambda col: pd.factorize(col)[0])

In [36]:
# Divide novamente após codificação
n_ref = len(df_ref)
X_ref = df_encoded.iloc[:n_ref].to_numpy()
X_cur = df_encoded.iloc[n_ref:].to_numpy()

In [37]:
# Detector de drift tabular
cd = TabularDrift(x_ref=X_ref, p_val=0.05, categories_per_feature={0: None, 1: None, 2: None})
preds = cd.predict(X_cur)

# Resultados
drift_detectado = preds['data']['is_drift']
p_values = preds['data']['p_val']
drift_features = [df_encoded.columns[i] for i, p in enumerate(p_values) if p < 0.05]

In [38]:
BASE_DIR = Path.cwd()  # Diretório atual onde o notebook está rodando
MLRUNS_PATH = BASE_DIR.parent / "mlruns"  # Sobe um nível e aponta para mlruns

print(MLRUNS_PATH)

d:\0Viviana\1 FIAP\5 fase\datathon\app\mlruns


In [39]:
# Define o tracking URI para MLflow
mlflow.set_tracking_uri(f"file:///{MLRUNS_PATH.as_posix()}")

In [40]:
# Log no MLflow
hoje = datetime.today()
timestamp = hoje.strftime("%Y%m%d_%H%M%S")

with mlflow.start_run(run_name=f"Drift_Alibi_{timestamp}"):
    mlflow.log_param("drift_detectado", drift_detectado)
    mlflow.log_param("features_com_drift", ", ".join(drift_features))
    mlflow.log_metric("num_features_drift", len(drift_features))

print(f"Drift detectado: {drift_detectado}")
print(f"Features com drift: {drift_features}")

Drift detectado: 1
Features com drift: ['cv_pt']
